# Logging in Python

Hi there!

In this notebook we will take a closer look @ Python logging. Logging is very important to get right! With it, you will not only be able to debug errors more easily. You will also be able to better analyse the performance of an application which is very valuable in production environments.

Below you will find multiple tips and tricks to level up your logging game!

## The logging module

Python has a neat, built in module that provides a flexible framework for logging messages.

In [1]:
import logging

### Logging classes

The package offers four components that are used during logging. Log events are passed between these components until they reach the final output.

* Loggers expose the interface that application code uses (creates the log records).
* Filters provide a fine grained system to control which log records to output.
* Handlers send the log records to the correct destination.
* Formatters specify the layout of log records.

The basic flow of log records between the components can be found in the image below.

![Log records flow](./img/logging_components.png "logging components")

### Loggers

When importing the `logging` package, a logger called `ROOT` exists by default. To create a new logger, use the following code. A nice rule to use when naming loggers is to use module-level loggers. This will make sure the logger tracks the hierarchy of the package, which make it very easy to know where event were logged, by the loggers name.

In [2]:
custom_logger = logging.getLogger('custom_logger')
module_level_logger = logging.getLogger(__name__)

If a logger has a parent logger (which is in most cases the `ROOT` logger), it will propagate the logged record to all parent loggers and when this happens, a single log record might be logged multiple times. To make sure this doesn't happen we can set the `propagate` flag to False.

In [3]:
custom_logger.propagate = False

#### The levels of logging

By default, the `logging` package offers five levels of logging. These levels indicate the severity of the logged events. The five levels, in order of increasing severity are:

* DEBUG (10): Problem diagnosing
* INFO (20): Confirmation messages of successful execution
* WARNING (30): Warn about unexpected situations
* ERROR (40): More serious syntax and logic problems
* CRITICAL (50): Report problems that result in crashes

To define the level of a certain log message, you can call the following corresponding methods on the logger you want to create the log records with:

In [4]:
custom_logger.debug('Debug message')
logging.info('Info message')
module_level_logger.warning('Warning message')
custom_logger.error('Error message')
logging.critical('Critical message')

Error message
CRITICAL:root:Critical message


In the log records we can notice multiple things. For one, we can clearly see which logger was used to log a certain event. We can also notice that the debug and info log records were not printed. The reason for this is that, by default, logging message with severity levels lower than `warning` or `30` will not be created. To lower and highten the severity level of records we want to see, use the following code.

In [5]:
custom_logger.setLevel('DEBUG')
module_level_logger.setLevel('ERROR')

custom_logger.debug('Debug message')
logging.info('Info message')
module_level_logger.warning('Warning message')
custom_logger.error('Error message')
logging.critical('Critical message')


Error message
CRITICAL:root:Critical message


By setting the level from our custom_logger to `DEBUG` and the level from our module_level_logger to `ERROR` we now get to see completely different log records. The custom logger can now output our debug event, and thus it shows up in the results. Our module-level logger however is logging a warning record but has it's output level set on `ERROR`. Thus, it will not create the record and we are not able to find it in our output.

#### Logging exceptions

One specific use case for logging is that we want to log certain errors in our code. If a certain exception occures, we want to know about it and be able to see it pop up in our logs. And we are in luck! The `logging` package provides a neat function for this (`logger.exception()`).

In [6]:
custom_logger.info('Started code.')
try:
    numbers = [5, 8, 9, 1, 'four']
    acc = sum(numbers)
except TypeError:
    custom_logger.exception('Calculating sum failed..')
custom_logger.info('Shut down code.')

Calculating sum failed..
Traceback (most recent call last):
  File "/tmp/ipykernel_5217/4156510778.py", line 4, in <module>
    acc = sum(numbers)
TypeError: unsupported operand type(s) for +: 'int' and 'str'


The function `.exception()` will automatically include the stack trace of the raised exception. It will create the log message with the `ERROR` severity level. You can only use this specific function from an exception handler!

### Filters

Filters can be used by both loggers and handlers for *next level* filtering, which is not provided by severity levels. To create a filter use the following code. From version 3.2 on, it is no longer necessary to subclass the `logging.Filter` class. You can now just create a function that will act as a filter.

In [7]:
class CustomFilter(logging.Filter):
    def filter(self, record):
        """To allow a certain record to be logged, return True.
        Return False to block a record from being logged.
        One can also modify the record in-place in this class method.
        """
        if record.name == 'custom_logger':
            return True
        return False

# After logging version 3.2
def custom_filter(record):
    """To allow a certain record to be logged, return True.
    Return False to block a record from being logged.
    One can also modify the record in-place in this class method.
    """
    if record.name == 'custom_logger':
        return True
    return False

In the custom filters created above, we check which logger created the log record. If it was the customer logger, we allow it to be logged. Otherwise we block the log record by returning `False`.

In [8]:
custom_logger.addFilter(custom_filter)
module_level_logger.addFilter(custom_filter)

custom_logger.critical('This record should get logged.')
module_level_logger.critical('This record should not get logged.')

custom_logger.removeFilter(custom_filter)
module_level_logger.removeFilter(custom_filter)

This record should get logged.


### Handlers

Handlers will dispatch log records to the handlers specified location. Both severity level and filters can be set on handlers (eg. only records are dispatched above severity level `INFO`). Loggers can have zero or multiple handlers. Handlers make it easy to send all log records to a log file, all log records with severity level `ERROR` or higher to stdout and all critical log records to an email address.

The `logging` library contains a few handler types that can be used. The ones to note are:

* StreamHandler - Sends log records to streams (file-like objects).
* FileHandler - Sends log records to disk files.
* QueueHandler - Sends log records to a queue, like those implemented in the queue or multiprocessing modules.
* SMTPHandler - Sends log records to a specified email address.

In [9]:
syslog = logging.StreamHandler()
syslog.setLevel('INFO')

filelog = logging.FileHandler('./logs/log_test.log', mode='a')
filelog.setLevel('ERROR')

if syslog not in custom_logger.handlers:
    custom_logger.addHandler(syslog)
    custom_logger.addHandler(filelog)

custom_logger.info('Started code.')
try:
    numbers = [5, 8, 9, 1, 'four']
    acc = sum(numbers)
except TypeError:
    custom_logger.exception('Calculating sum failed..')
custom_logger.info('Shut down code.')

Started code.
Calculating sum failed..
Traceback (most recent call last):
  File "/tmp/ipykernel_5217/4177430724.py", line 14, in <module>
    acc = sum(numbers)
TypeError: unsupported operand type(s) for +: 'int' and 'str'
Shut down code.


### Formatters

Formatters are used to configure the final order, structure and contents of a log record. One should pass a `fmt` or message format string to customize the format of the log records.

In [10]:
custom_logger.info('Log record without formatter.')

format_string = '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
date_format_string = '%d-%m-%Y %H:%M:%S'  # Defaults to %Y-%m-%d %H:%M:%S
custom_formatter = logging.Formatter(fmt=format_string, datefmt=date_format_string)
syslog.setFormatter(custom_formatter)  # A Format is linked to a handler!

custom_logger.info('Log record with formatter.')

Log record without formatter.
02-08-2021 13:48:15 - custom_logger - INFO - Log record with formatter.


You can also set the style the message format string is using. By default it uses the '%' style. However there are other styles, such as '{' or '$' that can be used. More information about this can be found [here](https://docs.python.org/3/howto/logging-cookbook.html#use-of-alternative-formatting-styles).

#### Adding contextual information

Sometimes, you will want to add custom contextual information into your log records. However one can not just add the variable to the format string of the Formatter, as this object will not recognize the variable. Luckily there are two seperate ways we can still accomplish this task. We could either use a Filter (as mentioned above, Filters can also be used to modify log records in-place.), or a LogAdapter.

A LogAdapter might not always work for your use case, so if it doesn't work with a LogAdapter, try to use a Filter.

In [11]:
extra = {'app_name':'Logging App'}

logger = logging.getLogger('Contextual-info-app-name-adapter')
logger.propagate = False
syslog = logging.StreamHandler()

format = '%(app_name)s: %(message)s'
formatter = logging.Formatter(format)
syslog.setFormatter(formatter)

logger.setLevel(logging.INFO)
logger.addHandler(syslog)
custom_adapter = logging.LoggerAdapter(logger, extra)
custom_adapter.info('Include contextual info')

Logging App: Include contextual info


In [12]:
def custom_filter(record):
    """To allow a certain record to be logged, return True.
    Return False to block a record from being logged.
    One can also modify the record in-place in this class method.
    """
    record.app_name = 'Logging App'
    return True

logger = logging.getLogger('Contextual-info-app-name-filter')
logger.propagate = False
logger.addFilter(custom_filter)
syslog = logging.StreamHandler()

format = '%(app_name)s: %(message)s'
formatter = logging.Formatter(format)
syslog.setFormatter(formatter)

logger.setLevel(logging.INFO)
logger.addHandler(syslog)
logger.info('Include contextual info')

Logging App: Include contextual info


### Set up logging with config

The `logging` package also provides easy ways to quickly set up a full logging system with all the logger, handlers, etc.. that are required. To do this you can use the `logging.config` module. It contains multiple functions that let you load in a predefined config in different formats to set up everything automatically.

* fileConfig - A logging config file. See `logging.conf` for an example.
* dictConfig - A dictionary of configuration information
  
To read and load in configurations, use the following code. Make sure to only import third party libraries after you have loaded in the logging config to avoid getting their logs!

In [19]:
import logging.config

logging.config.fileConfig('logging.conf')

# OR

LOGGING_CONFIG = {
    'version': 1,
    'disable_existing_loggers': True,
    'formatters': {
        'standard': {'format': '%(asctime)s [%(levelname)s] %(name)s: %(message)s'},
    },
    'handlers': {'default': {
        'level': 'INFO',
        'formatter': 'standard',
        'class': 'logging.StreamHandler',
        'stream': 'ext://sys.stdout', # Default is stderr
        }, 
    },
    'loggers': { '': { # root logger
            'handlers': ['default'],
            'level': 'WARNING',
            'propagate': False
        }, 'helpers.module': {
            'handlers': ['default'],
            'level': 'INFO',
            'propagate': False
        }, '__main__': { # if __name__ == '__main__'
            'handlers': ['default'],
            'level': 'DEBUG',
            'propagate': False
        },
    }
}
logging.config.dictConfig(LOGGING_CONFIG)

### Logging template

To set up a logging system quickly on a new project, use the logging template included in this repo in the `logging_template` folder!

## Resources to consult

* [Python Logging Cookbook](https://docs.python.org/3/howto/logging.html)
* [Medium blogposts](https://medium.com/search?q=python%20logging)